In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# Importing all datasets

a = pd.read_csv('data/data_1.csv', error_bad_lines=False)
b = pd.read_csv('data/data_2.csv', error_bad_lines=False)
c = pd.read_csv('data/data_3.csv', error_bad_lines=False) ; c.rename({'Field2':'Time'}, axis=1, inplace=True) # Bug caused while scraping
d = pd.read_csv('data/data_4.csv', error_bad_lines=False)

In [ ]:
# Combing all datasframes

df = pd.concat([a,b,c,d])
df.drop(columns=['Title_URL'], inplace=True)
df

In [ ]:
# Dropping all duplicates and NaN values

df.drop_duplicates(inplace=True)
df.dropna(inplace=True)
df

In [ ]:
# Pivot for better understanding
 
df = df.pivot_table(index='Title', columns='Field1', values='Time', aggfunc='first').reset_index()
df

In [ ]:
# Fixing and combining dates

df.Announced.info() # 95
df.Released.info() # 2306
df['Expected release'].info() # 92
df.fillna('', inplace=True)

def clean(col_1,col_2, key): # Source, Destination which will be fixed and has value in Source, Inclusive Pattern
    df1 = df.copy(deep=True)
    t = df1[[col_1,col_2]]
    t = t[t[col_1].str.contains(pat=key)]
    df.loc[t.index, [col_2]] = t[col_1]
    del df1, t
    return df

clean('Expected release','Released','20')
clean('Announced','Released','20')

## Feature Extraction

In [ ]:
df2 = df.copy()
df2['Brand'] = df['Title'].str.split(' ').str[0] # Brand
df2['Model'] = df['Title'].str.split(' ', 1).str[1] # Model
df2['Processor'] = df['Hardware'].str.split('\n').str[0] # CPU
df2['Memory'] = df['Hardware'].str.split('\n').str[1] # Memory
df2['OS'] = df['OS'].str.split('\n').str[0] # Display size
df2['Battery'] = df2['Battery'].str.replace(' mAh','') # change to regex keep numbers only
df2['Display_Size'] = df['Display'].str.split('\n').str[0] # Display size
df2['Display_Resolution'] = df['Display'].str.split('\n').str[1] # Display res
df2['Display_Refresh_Rate'] = df['Display'].str.split('\n').str[2] # Display Refresh Rate
df2['Primary_Camera'] = df['Camera'].str.split('\n').str[0] # Rear Camera
df2['Front_Camera'] = df['Camera'].str.split('\n').str[1] # Front Camera
df2['Primary_Storage'] = df['Storage'].str.split(',').str[0] # Internal Storage
df2['External_Storage'] = df['Storage'].str.split(',').str[1] # External Storage
df2.drop(columns=['Title','Display','Hardware','Camera','Storage'], inplace=True)


## Feature Formatting

In [ ]:
# Keeping numbers only
df2.Primary_Storage = df2.Primary_Storage.str.extract('(\d+)')
df2.Primary_Camera = df2.Primary_Camera.str.extract('(\d+)')
df2.Display_Refresh_Rate = df2.Display_Refresh_Rate.str.extract('(\d+)')

# Removing unnecessary characters and keeping the decimals
df2.Front_Camera = df2.Front_Camera.str.replace('MP VGA front', '')
df2.Front_Camera = df2.Front_Camera.str.replace('MP front', '')
df2.Front_Camera = df2.Front_Camera.str.replace('Other front', '')
df2.Front_Camera = df2.Front_Camera.str.replace('front', '')

df2.Display_Resolution = df2.Display_Resolution.str.replace('pixels', '')
df2.Display_Size = df2.Display_Size.str.replace('inches', '')
df2.Memory = df2.Memory.str.replace('GB RAM', '')
df2.Memory = df2.Memory.str.replace(' ','')

# Fixing spelling mistakes
df2.External_Storage = df2.External_Storage.str.replace('9microSDXC','microSDXC')
df2.External_Storage = df2.External_Storage.str.replace('not expandablemicroSDXC','microSDXC')

df2.rename({'Released':'Release_Date'}, axis=1, inplace=True)

# Replacing Yearly quater values for Date-time interpretation

df2.Release_Date = df2.Release_Date.str.replace('Q1','Feb')
df2.Release_Date = df2.Release_Date.str.replace('Q2','Jun')
df2.Release_Date = df2.Release_Date.str.replace('Q3','Sep')
df2.Release_Date = df2.Release_Date.str.replace('Q4','Nov')

In [ ]:
#df2.fillna('Not Specified', inplace=True)

df2[['Brand', 'Model', 'OS', 'Release_Date','Battery', 'Processor', 'Memory', 'Primary_Storage', 'External_Storage', 'Display_Size',
    'Display_Resolution','Display_Refresh_Rate','Primary_Camera','Front_Camera',]].to_csv('processed.csv')